# Tutorial: explain a tabular classifier through LORE

LORE provides interpretable explanations for the decisions made by machine learning models, particularly in situations where it might be challenging to explain the model's behavior using traditional methods. LORE is designed to offer explanations on a per-instance basis, meaning it provides insights into why a specific prediction was made for a particular data point. This is in contrast to global explanations that aim to provide an overview of a model's behavior across the entire dataset.


In this tutorial we will explain the reasoning of a random forest classifier when applied to an instance of the Adult Income dataset ( https://archive.ics.uci.edu/dataset/2/adult). This dataset contains census data from 1994 census database, the classification problem associated to such data involves the prediction of the annual income of a person, given a set of socio-economic caracteristics: 
- working class
- marital status
- occupation
- relationship
- race
- sex
- capital gain/loss
- native country 





### Dataset loading 

Lore libray has a module devoted to dataset handling. Here we use TabularDataset class to load a dataset from a csv file. A TabularDataset has a column that represent the target class (`class_name`) of the classification task  object, and has two main attributes: 
- `df` : a pandas dataframe representing the tabular data
- `descriptor` : a dictionary containing internal data related to the dataset. It is build when a TabularDataset is created, but it could also be edited. It is used by the next steps of LORE methodology to distinguish among numerical, categorical, ordinal and target features of the dataset.

In [32]:
from lore_sa.dataset import TabularDataset
import pandas as pd

dataset = TabularDataset.from_csv('test/resources/adult.csv', class_name = "class")
dataset.df.dropna(inplace = True)
dataset.df


2023-11-27 16:09:03,281 root         INFO     test/resources/adult.csv file imported


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [33]:
dataset.descriptor.keys()

dict_keys(['numeric', 'categorical', 'target'])

We drop some columns as part of a data preprocessing task. In this case, the descriptor of the dataset must be updated through the dedicated method


In [34]:
dataset.df.drop(['fnlwgt', 'education-num'], inplace=True, axis=1)
dataset.update_descriptor()

### One-hot encoding 

We apply one hot encoding to the dataset, in order to show how LORE handles encoding. The `encoder_decode` provides `TabularEnc` class to encode and decode a tabular dataset applying one hot encoding to the categorical features and label encoding to the target feature.

Tabular encoder has also a descriptor, that is derived from the originale Dataset descriptor and updated with the stats related to the encoded features.

In [35]:
#categorical features of our dataset
dataset.descriptor['categorical'].keys()

dict_keys(['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country'])

In [36]:
from lore_sa.encoder_decoder import TabularEnc
encoder = TabularEnc(dataset.descriptor)

encoded = []
for x in dataset.df.iloc:
    
    
    encoded.append(encoder.encode(x.values))
    
    


In [37]:
encoder.dataset_descriptor

{'numeric': {'age': {'index': 0,
   'min': 17,
   'max': 90,
   'mean': 38.437901995888865,
   'std': 13.134664776855985,
   'median': 37.0,
   'q1': 28.0,
   'q3': 47.0},
  'capital-gain': {'index': 8,
   'min': 0,
   'max': 99999,
   'mean': 1092.0078575691268,
   'std': 7406.346496683503,
   'median': 0.0,
   'q1': 0.0,
   'q3': 0.0},
  'capital-loss': {'index': 9,
   'min': 0,
   'max': 4356,
   'mean': 88.37248856176646,
   'std': 404.29837048637575,
   'median': 0.0,
   'q1': 0.0,
   'q3': 0.0},
  'hours-per-week': {'index': 10,
   'min': 1,
   'max': 99,
   'mean': 40.93123798156621,
   'std': 11.979984229274882,
   'median': 40.0,
   'q1': 40.0,
   'q3': 45.0}},
 'categorical': {'workclass': {'index': 1,
   'distinct_values': ['State-gov',
    'Self-emp-not-inc',
    'Private',
    'Federal-gov',
    'Local-gov',
    'Self-emp-inc',
    'Without-pay'],
   'count': {'State-gov': 1279,
    'Self-emp-not-inc': 2499,
    'Private': 22286,
    'Federal-gov': 943,
    'Local-gov': 20

Once we have an `TabularEnc` object, we can apply the `decode()` method to an encoded instance, in order to get the decoded instance.

In [38]:
encoder.decode(encoded[0])

array(['39', 'State-gov', 'Bachelors', 'Never-married', 'Adm-clerical',
       'Not-in-family', 'White', 'Male', '2174', '0', '40',
       'United-States', '<=50K'], dtype='<U21')

## The blackbox

LORE explains the reasoning of an input machine learning model (blackbox) that classifies an input instance of the dataset. Since this is a tutorial, we will create a blackbox from scratch, in order to explain it through LORE.

We will explain a blackbox that is taking one-hot-encoded instances as input. Hence, we create an encoded version of the original dataframe:

In [39]:
encoded_df = pd.DataFrame(encoded, columns = [encoder.encoded_features[i] for i in range(len(encoded[0]))])
encoded_df

,age,workclass=State-gov,workclass=Self-emp-not-inc,workclass=Private,workclass=Federal-gov,workclass=Local-gov,workclass=Self-emp-inc,workclass=Without-pay,education=Bachelors,education=HS-grad,...,native-country=Scotland,native-country=Trinadad&Tobago,native-country=Greece,native-country=Nicaragua,native-country=Vietnam,native-country=Hong,native-country=Ireland,native-country=Hungary,native-country=Holand-Netherlands,class
0,39,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,50,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,38,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,53,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,28,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30157,27,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30158,40,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
30159,58,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
30160,22,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


Here we create a Random Forest classifier, we train it on the Adult dataset, then we create an instance of Lore's BBox class based on the random forest just created.

In [40]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score

feature_names = [c for c in encoded_df.columns if c != 'class']
class_name = dataset.class_name
test_size= 0.3
random_state= 42

X_train, X_test, Y_train, Y_test = train_test_split(encoded_df[feature_names].values, encoded_df[class_name].values,
                                                    test_size=test_size,
                                                    random_state=random_state,
                                                    stratify=encoded_df[class_name].values)




# Train a random forest model

bb = RandomForestClassifier(n_estimators=20, random_state=random_state)
# bb = MLPClassifier(random_state=random_state)
bb.fit(X_train, Y_train)



# example of predictions
Y_pred = bb.predict(X_test)

print('Accuracy %.3f' % accuracy_score(Y_test, Y_pred))
print('F1-measure %.3f' % f1_score(Y_test, Y_pred))


from lore_sa.bbox import sklearn_classifier_bbox

bbox = sklearn_classifier_bbox.sklearnBBox(bb)



Accuracy 0.840
F1-measure 0.660


## Neighborhood generation

Given an element of the dataset, we can create an arbitrary number of synthetic instances. In this case we simulate the scenario where we only have a blackbox, a single element classified by such black box, and some limited information regarding features domains.

Here we use the `RandomGenerator` class to create 10000 elements starting from an element from the adult dataset. 

In [41]:
# random generation

features = [c for c in encoded_df.columns if c != dataset.class_name]

x = encoded_df[features].iloc[10].values

from lore_sa.neighgen.random import RandomGenerator

gen = RandomGenerator()

neighbour = gen.generate(x,10000, dataset.descriptor, onehotencoder = encoder)

neighbour.df

2023-11-27 16:09:47,717 root         WARNING  No target class is defined


,age,workclass=State-gov,workclass=Self-emp-not-inc,workclass=Private,workclass=Federal-gov,workclass=Local-gov,workclass=Self-emp-inc,workclass=Without-pay,education=Bachelors,education=HS-grad,...,native-country=Outlying-US(Guam-USVI-etc),native-country=Scotland,native-country=Trinadad&Tobago,native-country=Greece,native-country=Nicaragua,native-country=Vietnam,native-country=Hong,native-country=Ireland,native-country=Hungary,native-country=Holand-Netherlands
0,60.931878,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,29.769725,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,1,0,0,0,0
2,87.227799,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,84.346605,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,52.665892,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,54.823392,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
9996,32.927431,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
9997,66.103960,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
9998,62.478011,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0


We now use the black box we created before to classify all the instances of the new synthetic dataset 

In [42]:


neighbour.df['class'] = bbox.predict(neighbour.df[features])
neighbour.set_class_name('class')

## surrogate model

The `surrogate` submodule creates a classifier and provides the methods to extract its corresponding classification rules. Once trained the surrogate, we extract a set of rules that explains why the surrogate model classifies the instance in a certain way. 

In the following example, the instance `x` has been classified as an adult with income less than 50k. The surrogate model has used the features `capital-gain`, `capital-loss`, `marital-status`, and `native-country`. 

In [43]:
from lore_sa.surrogate import DecisionTreeSurrogate

surrogate = DecisionTreeSurrogate()


surrogate.train(neighbour.df[features].values , neighbour.df['class'] )

DecisionTreeClassifier()

In [44]:
x = encoded_df[features].iloc[10].values


rule = surrogate.get_rule(x, neighbour, encoder)

print (rule)

premises:
capital-gain <= 5026.990234375
capital-loss <= 1742.9114990234375
marital-status = Married-civ-spouse
occupation != Machine-op-inspct 
consequence: class = <=50K


In [45]:
crules, deltas = surrogate.get_counterfactual_rules(x=x, class_name = 'class', feature_names=features, 
                                                    neighborhood_dataset = neighbour, encoder = encoder)

premises:
capital-gain > 8926.58544921875
occupation != Other-service
age > 26.59445858001709
relationship != Own-child
capital-loss > 2344.1400146484375
hours-per-week <= 40.72099304199219
marital-status != Divorced
education != HS-grad
race != Black
native-country != Honduras
occupation != Exec-managerial
native-country != China
native-country != Philippines
native-country != Laos
native-country != Puerto-Rico 
consequence: class = >50K [<lore_sa.rule.Expression object at 0x7f32e100bfd0>, <lore_sa.rule.Expression object at 0x7f32e100be80>, <lore_sa.rule.Expression object at 0x7f32e100bb20>]
premises:
capital-gain <= 97655.5234375
capital-gain > 10999.65771484375
occupation != Other-service
age > 30.110142707824707
relationship != Own-child
capital-loss <= 2344.1400146484375
relationship = Other-relative
marital-status != Never-married
hours-per-week <= 34.31813049316406
education != 1st-4th
native-country != Peru
workclass != Private
education != Some-college
native-country != Taiwan

In [46]:
for r in crules:
    print(r)

premises:
capital-gain <= 6965.611572265625
capital-loss <= 2409.8294677734375
marital-status != Married-civ-spouse
age <= 87.45146560668945
native-country != Greece
occupation != Exec-managerial
native-country != Trinadad&Tobago
native-country != Columbia
native-country != Philippines 
consequence: class = <=50K
